In [1]:
from newscatcher import Newscatcher
import pandas as pd
from datetime import datetime

In [2]:
all_headlines_csv = 'data/google-news-headlines.csv'
all_df_headlines = pd.read_csv(all_headlines_csv)

In [3]:
now = datetime.utcnow().strftime('%Y-%m-%d %H:%M')
now

'2020-02-27 22:05'

In [4]:
# get the current headlines
ns = Newscatcher('news.google.com')
df_latest = pd.DataFrame({'datetime': now, 'headline': ns.headlines})

In [5]:
df_latest

,datetime,headline
0,2020-02-27 22:05,How bad is the coronavirus-sparked stock-marke...
1,2020-02-27 22:05,Shooter at Milwaukee Molson Coors had a long-r...
2,2020-02-27 22:05,Coronavirus Mortality Rate: How COVID-19 Fatal...
3,2020-02-27 22:05,The 31 wildest lines from Donald Trump's self-...
4,2020-02-27 22:05,Pence Will Control All Coronavirus Messaging F...
5,2020-02-27 22:05,Congress Nears Bipartisan Funding Plan To Addr...
6,2020-02-27 22:05,CDC: Possible case of 'community spread' coron...
7,2020-02-27 22:05,"Lori Vallow, mother of missing Idaho kids, den..."
8,2020-02-27 22:05,Poll: Biden crushing Sanders in South Carolina...
9,2020-02-27 22:05,De Blasio rips Bloomberg in Hannity interview:...


In [6]:
# merge with the old one
all_df_headlines = all_df_headlines.append(df_latest, sort=True)


In [7]:
all_df_headlines.drop_duplicates(['headline'], inplace=True)

In [8]:
# save all headlines  to file
all_df_headlines.to_csv(all_headlines_csv, index=False)

In [9]:
# TODO count those with coronavirus